# Projeto Final - Sistema de Controle Financeiro

## Grupo 1
* Jose Marchezi,
* Renato Freitas,
* Nicolas Evilasio,
* Alê,
* Rafael Gustavo Leite
* Mayra Juliana Pereira Alves


# Nome Criativo para a aplicação

O aplicativo utiliza dois arquivos como base de dados:

- investimentos.csv

- movimentacoes.csv



Importar modulos

In [1]:
from src.utils.funcoes import *

c:\Users\nicol\OneDrive - COMERCIO DE PERFUMARIA GINSENG LTDA\Python\SantanderCoders\Projeto\Santander-Coders-2023-ED\projeto_LPII


Parametros "fixos" de configuração:
* PATH: informa o diretório onde os dados serão armazenados
* TAXA_INVESTIMENTO: Valor utilizado no calculo do rentimento.

In [2]:
PATH = "./database"
TAXA_INVESTIMENTO = 0.01

## Iniciar Base de dados

Criamos uma função para incluir e gerar a nosa base de dados. Essa base é constituída de dois arquivos: 

* "investimentos.csv"
  - Armazena as entradas de investimentos
* "movimentacoes.csv"
  - Armazena as entradas de receita e despesas da conta 

Essa função oferece ao usuário duas opções para inclusão manual dos dados:

1. usuário informa a data (no formato ano-mes-dia); o valor; e tipo (investimento, receita, ou despesa).

2. a entradas serão lidas de um arquivo .csv, o usuario deve informar o nome desse arquivo.


In [3]:
incluir_registros_base_dados(taxa=TAXA_INVESTIMENTO, path=PATH)

Como você deseja incluir os registros?
              
            1. Adicionar manualmente
            2. Importar Base CSV
            9. Retornar ao menu principal
            0. Sair do programa
              
Registro inserido com sucesso.
Como você deseja incluir os registros?
              
            1. Adicionar manualmente
            2. Importar Base CSV
            9. Retornar ao menu principal
            0. Sair do programa
              


## Deletar registro

In [4]:
deletar_registro(indice=3, tipo="despesa", database_path=PATH)

Registro {'id': '0000004', 'tipo': 'despesa', 'valor': '11', 'ano': '2023', 'mes': '4', 'dia': '9'} deletado com sucesso!


## Agrupar dados

In [5]:
movimentacoes = read_csv(f"{PATH}/movimentacoes.csv")
investimentos = read_csv(f"{PATH}/investimentos.csv")


print(agrupar_movimentacoes(movimentacoes, agrupar_por="tipo"))
print(agrupar_movimentacoes(movimentacoes, agrupar_por="ano"))

print(agrupar_movimentacoes(investimentos, agrupar_por="ano"))

{'receita': 16041.0, 'despesa': -66652.0}
{'valor_ano_2023': -19219.0, 'valor_ano_2024': -12391.0, 'valor_ano_2020': -20000.0, 'valor_ano_2013': 999.0}
{'valor_ano_2023': 3344479219119501.5, 'valor_ano_2024': 14823.96, 'valor_ano_2020': 2.2002539656363495e+20, 'valor_ano_2000': 2.200220533137737e+20}


## Atualizar registro

In [6]:
movimentacoes = read_csv(f"{PATH}/movimentacoes.csv")
movimentacoes[3]

{'id': '0000005',
 'tipo': 'receita',
 'valor': '5423.0',
 'ano': '2023',
 'mes': '6',
 'dia': '14'}

In [7]:
atualizar_registro(dia=9, 
                    mes=4, 
                    ano=2023, 
                    valor=11, 
                    tipo='despesa', 
                    database_path=PATH)

TypeError: list indices must be integers or slices, not NoneType

In [8]:
movimentacoes = read_csv(f"{PATH}/movimentacoes.csv")
movimentacoes[3]

{'id': '0000005',
 'tipo': 'receita',
 'valor': '5423.0',
 'ano': '2023',
 'mes': '6',
 'dia': '14'}

In [9]:
investimentos = read_csv(f"{PATH}/investimentos.csv")
investimentos[3]

{'id': '0000004',
 'tipo': 'investimento',
 'capital': '1',
 'taxa': '0.01',
 'ano': '2023',
 'mes': '5',
 'dia': '28',
 'montante': '17064.33',
 'rendimento': '7187.09'}

In [10]:

atualizar_registro(dia=28, 
                    mes=5, 
                    ano=2023, 
                    valor=1, 
                    tipo='investimento', 
                    database_path=PATH,
                    taxa=TAXA_INVESTIMENTO)

In [11]:
investimentos = read_csv(f"{PATH}/investimentos.csv")
investimentos[3]

{'id': '0000004',
 'tipo': 'investimento',
 'capital': '1',
 'taxa': '0.01',
 'ano': '2023',
 'mes': '5',
 'dia': '28',
 'montante': '12077.87',
 'rendimento': '0.0'}

## Exportar os dados

In [4]:
movimentacoes = read_csv(f"{PATH}/movimentacoes.csv")
exportar_relatorio_json(movimentacoes, formato='json', nome_arquivo='relatorio_movimentacoes')
investimentos = read_csv(f"{PATH}/investimentos.csv")
exportar_relatorio_json(investimentos, formato='json', nome_arquivo='relatorio_investimentos')

In [5]:
investimentos = read_csv(f"{PATH}/investimentos.csv")

In [6]:
investimentos

[{'id': '0000001',
  'tipo': 'investimento',
  'capital': '100.0',
  'taxa': '0.01',
  'ano': '2024',
  'mes': '1',
  'dia': '1',
  'montante': '100.0',
  'rendimento': '0'},
 {'id': '0000002',
  'tipo': 'investimento',
  'capital': '100.0',
  'taxa': '0.01',
  'ano': '2023',
  'mes': '1',
  'dia': '1',
  'montante': '200.0',
  'rendimento': '-97.353'},
 {'id': '0000003',
  'tipo': 'investimento',
  'capital': '100.0',
  'taxa': '0.01',
  'ano': '2024',
  'mes': '1',
  'dia': '2',
  'montante': '202.65',
  'rendimento': '7432.254'},
 {'id': '0000004',
  'tipo': 'investimento',
  'capital': '2000.0',
  'taxa': '0.01',
  'ano': '2023',
  'mes': '1',
  'dia': '31',
  'montante': '9634.9',
  'rendimento': '-195.492'}]